# count_vectorizer_sentiment_analysis

In [1]:
import pandas as pd
import numpy as np
import os
import re
import random
import pandas as pd

In [6]:
import pandas as pd

chunk_size = 1000
chunks = pd.read_csv(r"D:\Generative AI\aws_review_sofware_dataset.csv", sep=',', chunksize=chunk_size)

# Get the first chunk and access its columns
df = next(chunks)
print(df.columns)


Index(['Unnamed: 0', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'vote', 'image'],
      dtype='object')


In [7]:
df.columns

Index(['Unnamed: 0', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'vote', 'image'],
      dtype='object')

In [10]:
df["words"]="default value"
df["sentences"]="default value"


for i in range(df.shape[0]): 
    df.at[i,"words"]= list("")
    df.at[i,"sentences"] = list("")

Purpose: This initializes two new columns, words and sentences, in the DataFrame df. 

Every cell in these columns is initially set to the string "default value".

This loop iterates over the rows of the DataFrame. df.shape[0] gives the number of rows in the DataFrame.

This modifies the values of the words and sentences columns for the row at index i.

df.at[i, "words"]: Accesses the value in the words column at row index i.

list(""): Converts an empty string ("") into a list. The result is an empty list [].


In [11]:
from nltk.tokenize import sent_tokenize

#### The line from nltk.tokenize import sent_tokenize imports the sent_tokenize function from the Natural Language Toolkit (NLTK) library. This function is used to split a given text into sentences.
# What Does sent_tokenize Do?
sent_tokenize:

Breaks a paragraph or block of text into a list of sentences.


In [ ]:
import nltk
nltk.download('punkt')

The command nltk.download('punkt') is used to download the Punkt tokenizer models in the NLTK library. These models are essential for tokenizing text into sentences or words and are pre-trained to recognize sentence boundaries in various languages.

What is Punkt?

Punkt is an unsupervised machine learning algorithm for tokenizing text. It was developed by Martin Kiss and is widely used for identifying:

1)Sentence boundaries (sent_tokenize).

2)Word boundaries (word_tokenize).

Purpose of nltk.download('punkt')

It downloads the data files required for sentence and word tokenization.

These files contain pre-trained models that help recognize:

Sentence-ending punctuation (e.g., ., !, ?).

Abbreviations (e.g., Dr., e.g., i.e.).

Edge cases like quotes or ellipses.

In [12]:
for i in range(df.shape[0]):
    l1= sent_tokenize(str(df.loc[i,"reviewText"]))
    df.at[i,"sentences"]=l1

This code iterates through each row in a DataFrame df, extracts the text from the column "reviewText", tokenizes it into sentences using sent_tokenize, and assigns the resulting list of sentences to the column "sentences"

In [18]:
#!pip install pywsd

In [16]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Sreedhar\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_eng.zip.


True

### What is the Averaged Perceptron Tagger?

Purpose: It is a machine learning-based POS tagger provided by NLTK that assigns parts of speech (e.g., noun, verb, adjective) to each word in a sentence.

Usage: It works well for English text and is pre-trained on annotated corpora.


In [17]:
from pywsd.utils import lemmatize_sentence

The line from pywsd.utils import lemmatize_sentence imports the lemmatize_sentence function from the PyWSD (Python Word Sense Disambiguation) library. This function is used for lemmatizing a sentence, which means reducing each word to its base or root form, while considering its part of speech.

What is Lemmatization?

Lemmatization reduces words to their dictionary (base) form, known as the lemma.

Unlike stemming, lemmatization is more context-aware and uses a vocabulary and morphological analysis.

Example: "running" → "run"

Example: "better" → "good" (Lemmatization considers meaning and context).

In [19]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Sreedhar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

The command nltk.download('wordnet') downloads the WordNet lexical database in NLTK. This resource is essential for various Natural Language Processing (NLP) tasks, including lemmatization and word sense disambiguation.

#### What is WordNet?
WordNet is a large, freely available lexical database of the English language.

It groups words into sets of synonyms called synsets and provides short definitions and usage examples.
#### WordNet helps with:
Understanding word meanings (semantics).

Finding synonyms, antonyms, and hyponyms.

Lemmatization: Reducing a word to its dictionary form based on its meaning.

In [20]:
for k in range(df.shape[0]):
    df.at[k,"words"]=list("")
    for j in range(len(df.loc[k,"sentences"])):
        df.at[k,"words"].extend(lemmatize_sentence(df.loc[k,"sentences"][j]))

In [21]:
df["words_sentences"] = "default"

In [22]:
import functools
for k in range(df.shape[0]):
    df.loc[k,"words_sentences"]=functools.reduce(lambda a,b:( str(a)+str(" ")+str(b)),df.loc[k,"words"])

In [23]:
from sklearn.feature_extraction.text import  CountVectorizer

In [24]:
df1=df

no_features = 1000
tf_vectorizer = CountVectorizer( max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(df1.words_sentences)

In [25]:
df_x=pd.DataFrame(tf.A, columns=tf_vectorizer.get_feature_names_out())

In [26]:
df_y=df["verified"]

In [27]:

from sklearn.preprocessing import LabelEncoder

In [28]:

le=LabelEncoder()

In [29]:
df_y_1=pd.DataFrame(df_y)

In [30]:
df_y_enc=df_y_1.apply(le.fit_transform)

In [31]:
df_y_enc.columns

Index(['verified'], dtype='object')

In [32]:
df_y_enc.head(5)

,verified
0,1
1,1
2,1
3,1
4,0


In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# Random Forest Classifier
rf = RandomForestClassifier(n_estimators=500, random_state=42)
rf.fit(df_x,df_y_enc)

# Accuracy
accuracy_rf = rf.score(df_x,df_y_enc)
print(f"Random Forest Accuracy: {accuracy_rf * 100:.2f}%")


C:\Users\Sreedhar\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Random Forest Accuracy: 100.00%


In [34]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(df_x,df_y_enc)

# Compute accuracy
accuracy_nb = nb.score(df_x,df_y_enc)
print(f"Naive Bayes Accuracy: {accuracy_nb * 100:.2f}%")


Naive Bayes Accuracy: 89.00%


C:\Users\Sreedhar\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [35]:
from sklearn.ensemble import GradientBoostingClassifier

In [36]:
GBC=GradientBoostingClassifier(n_estimators=100)

In [37]:
gb_c=GBC.fit(df_x,df_y_enc)

C:\Users\Sreedhar\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [38]:
gbc_score=GBC.score(df_x,df_y_enc)
print(f"gbc_score: {gbc_score* 100:.2f}%")

gbc_score: 96.60%
